In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import os
from rdflib import Graph
import rdflib as rdf

c:\Users\luisa\anaconda3\envs\aifb\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def read_all_pickle_without_ending(path):
    """
    Read all pickle files in a directory and return a list of the dataframes.
    """
    files = os.listdir(path)
    files = [file for file in files if file.endswith(".pkl")]
    files.sort()
    dfs = []
    for file in files:
        dfs.append(pd.read_pickle(path + file))
    return dfs

def readallcsvinpath(path):
    """
    Read all csv files in a directory and return a list of the dataframes.
    """
    fil = os.listdir(path)
    fil = [file for file in fil if file.endswith(".csv")]
    fil.sort()
    dfs = []
    for file in fil:
        dfs.append(pd.read_csv(path + file))
    return dfs
def get_nameincolfor_index(df,id_list):
    """
    # df: dataframe
    # id_list: list of ids
    # return: list of names of ids
    """
    name_list=[]
    for i in id_list:
        name_list.append(df[df.index==i][0].values[0])
    return name_list
def cut_string(list):
    for i in range(len(list)):
        list[i] = list[i].split('mov')[1]
    return list

In [5]:
graph = rdf.Graph()
graph.parse('data/IMDB/imdb_graph.nt', format='nt')

<Graph identifier=Nbbd54bfcfd3448ea9bad87a72c392137 (<class 'rdflib.graph.Graph'>)>

In [6]:
files = read_all_pickle_without_ending('out/IMDB/RGAT_no_emb/')
csv = readallcsvinpath('out/IMDB/RGAT_no_emb/')
ids = [1199,2613,2048,1469]
data = torch.load('out/IMDB/RGAT_no_emb/pred_before.pt')
test_file = pd.read_csv('data/IMDB/test_Set_new.csv', sep=',')
nodes = pd.DataFrame(files[1]) 
movies = pd.read_csv('data/IMDB/tog.csv', sep=',')
gen = movies[['Drama', 'Comedy', 'Romance', 'Thriller']]
new = gen.dot(gen.columns)
movies['genres'] = new
movies[['actor','actress','director']] = movies[['actor','actress','director']].astype(int)
job = movies[['actor','actress','director']]
movies['job'] = job.dot(job.columns)
movies

,nconst,actor,primaryName,actress,knownForTitles,director,tconst,titleType,originalTitle,startYear,Drama,Comedy,Romance,Thriller,genres,job
0,nm0007562,1,Imad Ab on Khaled,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
1,nm0196878,1,Emad Daher,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
2,nm0251614,1,Husein Abu Eid,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
3,nm0259524,1,Hans Erlandsson,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
4,nm0323487,1,Ibrahim Gneim,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,nm9324389,1,Gunnel Holzhausen,0,tt0002514,0,tt0002514,movie,De svarta maskerna,1912,1,0,0,0,Drama,actor
3088,nm9507151,0,Frl. Stoike,1,tt0001914,0,tt0001914,movie,Die Sünden der Väter,1913,1,0,0,0,Drama,actress
3089,nm9843327,1,N. Moroz,0,tt0083794,0,tt0083794,movie,Davay pozhenimsya,1982,0,0,1,0,Romance,actor
3090,nm9843328,1,V. Kalashnikov,0,tt0083794,0,tt0083794,movie,Davay pozhenimsya,1982,0,0,1,0,Romance,actor


In [7]:
test = get_nameincolfor_index(nodes,ids)
res = cut_string(test)
overview = movies[movies.knownForTitles.isin(res)][['primaryName','originalTitle','startYear','genres','job', 'knownForTitles','nconst']]
dicto = {res[i]: ids[i] for i in range(len(res))}
overview['id'] = overview.knownForTitles.map(dicto)
test_idx = nodes[nodes[0].isin(test_file.movie)]
test_pred = data[test_idx.index]
overview

,primaryName,originalTitle,startYear,genres,job,knownForTitles,nconst,id
319,Ruth Allen,Down to Earth,1917,Comedy,actress,tt0007873,nm0021020,1469
320,William H. Keith,Down to Earth,1917,Comedy,actor,tt0007873,nm0445318,1469
2028,Charles Logan,Journey Into Fear,1975,Thriller,actor,tt0073213,nm0517526,2048
2030,John Loughney,Beyond the Time Barrier,1960,Romance,actor,tt0053651,nm0521774,2613
2031,Tom Ravick,Beyond the Time Barrier,1960,Romance,actor,tt0053651,nm0712447,2613
2032,Malcolm Thompson,Beyond the Time Barrier,1960,Romance,actor,tt0053651,nm3911635,2613
2186,Jose Portullano,Um Chá nas Nuvens,1917,Comedy,actor,tt0007794,nm0692545,1199
2187,Miguel Portullano,Um Chá nas Nuvens,1917,Comedy,actor,tt0007794,nm0692546,1199


In [8]:
labels = test_file['genre'].astype('category').cat.codes 
test_file['label'] = labels.values
test_file['pred_label'] =test_pred.argmax(dim=-1).tolist()
test_file

,movie,id,genre,label,pred_label
0,http://imdb.org/movtt0004902,175,http://imdb.org/genDrama,1,2
1,http://imdb.org/movtt0093897,702,http://imdb.org/genThriller,3,3
2,http://imdb.org/movtt0009482,390,http://imdb.org/genComedy,0,3
3,http://imdb.org/movtt0040801,452,http://imdb.org/genThriller,3,1
4,http://imdb.org/movtt0004470,131,http://imdb.org/genDrama,1,3
5,http://imdb.org/movtt0004956,186,http://imdb.org/genDrama,1,3
6,http://imdb.org/movtt0107186,823,http://imdb.org/genRomance,2,0
7,http://imdb.org/movtt0004363,121,http://imdb.org/genComedy,0,3
8,http://imdb.org/movtt0099380,741,http://imdb.org/genThriller,3,0
9,http://imdb.org/movtt0003772,68,http://imdb.org/genDrama,1,0


In [9]:
topmax = csv[1].sort_values(by=['max_nodes'], ascending=False)
topmax.head()

,Unnamed: 0,Unnamed: 1,tensor_nodes,max_nodes,pos_max_nodes,min_nodes,pos_min_nodes
33,33,"tensor(1469, device='cuda:7')","tensor(indices=tensor([[ 650, 1127, 1469, 2761...",1.044495,1127,-1.006582,1469
13,13,"tensor(1199, device='cuda:7')","tensor(indices=tensor([[ 650, 769, 1199, 1470...",0.906776,2761,-0.661224,1199
32,32,"tensor(2048, device='cuda:7')","tensor(indices=tensor([[ 927, 1769, 2048, 2721...",0.862930,1769,-0.076685,2048
27,27,"tensor(2079, device='cuda:7')","tensor(indices=tensor([[ 277, 408, 2079, 2684...",0.818762,277,-0.013352,2079
6,6,"tensor(2178, device='cuda:7')","tensor(indices=tensor([[ 680, 880, 1819, 2178...",0.809685,880,-0.041777,2178


In [10]:
genres_dict = {'0': 'Comedy', '1': 'Drama', '2': 'Romance', '3': 'Thriller'}
for i in topmax[:10].index:
    node_id = topmax['Unnamed: 1'][i].split(',')[0].split('(')[1]
    max_node = topmax.max_nodes[i]
    pos_max_node = topmax.pos_max_nodes[i]
    pred_node_info = overview[overview['id']== int(node_id)]
    uri_pred = nodes[nodes.index == int(node_id)][0].values[0]
    uri_best = nodes[nodes.index == int(pos_max_node)][0].values[0]
    print(uri_pred)
    try:
        job = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['job'].values[0]
        name_best = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['primaryName'].values[0]
    except:
        print('uri_best')
        name_best = movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'].values[0]
        job = 'Year'
        #print('nam:',name_best, movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'] )
    
    mov_title = movies[movies.knownForTitles == uri_pred.split('mov')[1]]['originalTitle'].values[0]
    print('+++++++ STARTING WITH NODE ', mov_title, '+++++++++++++')
    pred_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['pred_label'].values[0])]
    real_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['label'].values[0])]
    print('The movie ', mov_title, 'has the predicted genre ', pred_genre, 'and the real genre ', real_genre, 'because of node ',job, name_best,'.')

    for s, p, o in graph:
        if s == rdf.term.URIRef(uri_pred):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o ==  rdf.term.URIRef(uri_pred):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
        if s ==  rdf.term.URIRef(uri_best):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o == rdf.term.URIRef(uri_best):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            elif job == 'Year':
                print(s)
                #s = 'movie ' + movies[movies.knownForTitles == ('tt' + s.split('/')[-1].split('tt')[1])]['originalTitle'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    print('\n')

http://imdb.org/movtt0007873
+++++++ STARTING WITH NODE  Down to Earth +++++++++++++
The movie  Down to Earth has the predicted genre  Thriller and the real genre  Comedy because of node  actor William H. Keith .
The movie  Down to Earth is the following property of: acted_in William H. Keith
The movie  Down to Earth has the following properties: has_actor William H. Keith
The movie  Down to Earth has the following properties: published_in year1917
The  actor William H. Keith is the following property of: collabs_with Marie Hubert Frohman
The  actor William H. Keith is the following property of: preferred_by genComedy
The  actor William H. Keith has the following properties: collabs_with Marie Hubert Frohman


http://imdb.org/movtt0007794
+++++++ STARTING WITH NODE  Um Chá nas Nuvens +++++++++++++
The movie  Um Chá nas Nuvens has the predicted genre  Comedy and the real genre  Comedy because of node  director Marie Hubert Frohman .
The  director Marie Hubert Frohman is the following pr

In [11]:
topmin = csv[1].sort_values(by=['min_nodes'], ascending=True)
topmin.head()

,Unnamed: 0,Unnamed: 1,tensor_nodes,max_nodes,pos_max_nodes,min_nodes,pos_min_nodes
33,33,"tensor(1469, device='cuda:7')","tensor(indices=tensor([[ 650, 1127, 1469, 2761...",1.044495,1127,-1.006582,1469
13,13,"tensor(1199, device='cuda:7')","tensor(indices=tensor([[ 650, 769, 1199, 1470...",0.906776,2761,-0.661224,1199
32,32,"tensor(2048, device='cuda:7')","tensor(indices=tensor([[ 927, 1769, 2048, 2721...",0.862930,1769,-0.076685,2048
8,8,"tensor(877, device='cuda:7')","tensor(indices=tensor([[ 422, 775, 877, 2602...",0.762039,775,-0.042568,877
6,6,"tensor(2178, device='cuda:7')","tensor(indices=tensor([[ 680, 880, 1819, 2178...",0.809685,880,-0.041777,2178


In [12]:
genres_dict = {'0': 'Comedy', '1': 'Drama', '2': 'Romance', '3': 'Thriller'}
for i in topmin[:10].index:
    node_id = topmin['Unnamed: 1'][i].split(',')[0].split('(')[1]
    max_node = topmin.min_nodes[i]
    pos_max_node = topmin.pos_min_nodes[i]
    pred_node_info = overview[overview['id']== int(node_id)]
    uri_pred = nodes[nodes.index == int(node_id)][0].values[0]
    uri_best = nodes[nodes.index == int(pos_max_node)][0].values[0]
    print(uri_pred)
    try:
        job = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['job'].values[0]
        name_best = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['primaryName'].values[0]
    except:
        print('uri_best')
        name_best = movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'].values[0]
        job = 'Year'
        #print('nam:',name_best, movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'] )
    
    mov_title = movies[movies.knownForTitles == uri_pred.split('mov')[1]]['originalTitle'].values[0]
    print('+++++++ STARTING WITH NODE ', mov_title, '+++++++++++++')
    pred_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['pred_label'].values[0])]
    real_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['label'].values[0])]
    print('The movie ', mov_title, 'has the predicted genre ', pred_genre, 'and the real genre ', real_genre, 'because of node ',job, name_best,'.')

    for s, p, o in graph:
        if s == rdf.term.URIRef(uri_pred):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o ==  rdf.term.URIRef(uri_pred):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
        if s ==  rdf.term.URIRef(uri_best):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o == rdf.term.URIRef(uri_best):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            elif job == 'Year':
                print(s)
                #s = 'movie ' + movies[movies.knownForTitles == ('tt' + s.split('/')[-1].split('tt')[1])]['originalTitle'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    print('\n')

http://imdb.org/movtt0007873
uri_best
+++++++ STARTING WITH NODE  Down to Earth +++++++++++++
The movie  Down to Earth has the predicted genre  Thriller and the real genre  Comedy because of node  Year 1917 .
The movie  Down to Earth is the following property of: acted_in William H. Keith
William H. Keith
The  Year 1917 is the following property of: acted_in William H. Keith
The movie  Down to Earth has the following properties: has_actor William H. Keith
The  Year 1917 has the following properties: has_actor William H. Keith
The movie  Down to Earth has the following properties: published_in year1917
The  Year 1917 has the following properties: published_in year1917


http://imdb.org/movtt0007794
uri_best
+++++++ STARTING WITH NODE  Um Chá nas Nuvens +++++++++++++
The movie  Um Chá nas Nuvens has the predicted genre  Comedy and the real genre  Comedy because of node  Year 1917 .
The movie  Um Chá nas Nuvens has the following properties: published_in year1917
The  Year 1917 has the fol

In [2]:
files = read_all_pickle_without_ending('out/IMDB/RGCN_no_emb/')
csv = readallcsvinpath('out/IMDB/RGCN_no_emb/')
ids = [268,1493,2032,1130,321,2060]
data = torch.load('out/IMDB/RGCN_no_emb/pred_before.pt')
test_file = pd.read_csv('data/IMDB/test_Set_new.csv', sep=',')
nodes = pd.DataFrame(files[1]) 
movies = pd.read_csv('data/IMDB/tog.csv', sep=',')
gen = movies[['Drama', 'Comedy', 'Romance', 'Thriller']]
new = gen.dot(gen.columns)
movies['genres'] = new
movies[['actor','actress','director']] = movies[['actor','actress','director']].astype(int)
job = movies[['actor','actress','director']]
movies['job'] = job.dot(job.columns)
movies

NameError: name 'read_all_pickle_without_ending' is not defined

In [22]:
test = get_nameincolfor_index(nodes,ids)
res = cut_string(test)
overview = movies[movies.knownForTitles.isin(res)][['primaryName','originalTitle','startYear','genres','job', 'knownForTitles','nconst']]
dicto = {res[i]: ids[i] for i in range(len(res))}
overview['id'] = overview.knownForTitles.map(dicto)
test_idx = nodes[nodes[0].isin(test_file.movie)]
test_pred = data[test_idx.index]
overview

,primaryName,originalTitle,startYear,genres,job,knownForTitles,nconst,id
688,Gertrude Barrett,Without Hope,1914,Comedy,actress,tt0004828,nm0057141,321
751,Auge Becker,Peck's Bad Girl,1918,Comedy,actor,tt0009482,nm0065348,1493
752,F.G. Patton,Peck's Bad Girl,1918,Comedy,actor,tt0009482,nm0666559,1493
1815,James Hester,Mrs. Black Is Back,1914,Comedy,actor,tt0004363,nm0381677,2032
1958,Marian Kinmaird,"Alma, Where Do You Live?",1917,Comedy,actress,tt0007628,nm0455674,1130
2368,Marion Whitney,The Apaches of Paris,1915,Drama,actress,tt0004902,nm0926330,268
3005,Si-Fun Yeung,Jia ri qing wei le,1993,Romance,actress,tt0108605,nm5767799,2060


In [23]:
labels = test_file['genre'].astype('category').cat.codes 
test_file['label'] = labels.values
test_file['pred_label'] =test_pred.argmax(dim=-1).tolist()
test_file

,movie,id,genre,label,pred_label
0,http://imdb.org/movtt0004902,175,http://imdb.org/genDrama,1,3
1,http://imdb.org/movtt0093897,702,http://imdb.org/genThriller,3,3
2,http://imdb.org/movtt0009482,390,http://imdb.org/genComedy,0,1
3,http://imdb.org/movtt0040801,452,http://imdb.org/genThriller,3,2
4,http://imdb.org/movtt0004470,131,http://imdb.org/genDrama,1,1
5,http://imdb.org/movtt0004956,186,http://imdb.org/genDrama,1,1
6,http://imdb.org/movtt0107186,823,http://imdb.org/genRomance,2,1
7,http://imdb.org/movtt0004363,121,http://imdb.org/genComedy,0,3
8,http://imdb.org/movtt0099380,741,http://imdb.org/genThriller,3,3
9,http://imdb.org/movtt0003772,68,http://imdb.org/genDrama,1,1


In [25]:
topmax = csv[1].sort_values(by=['max_nodes'], ascending=False)
topmax

,Unnamed: 0,Unnamed: 1,tensor_nodes,max_nodes,pos_max_nodes,min_nodes,pos_min_nodes
0,0,"tensor(268, device='cuda:7')","tensor(indices=tensor([[ 110, 211, 229, 239...",1.991685,1428,-1.002726,268
2,2,"tensor(1493, device='cuda:7')","tensor(indices=tensor([[ 44, 129, 197, 247...",1.695369,401,-1.093454,2448
7,7,"tensor(2032, device='cuda:7')","tensor(indices=tensor([[ 6, 12, 35, 42...",1.574197,401,-0.801227,1996
13,13,"tensor(741, device='cuda:7')","tensor(indices=tensor([[ 121, 322, 394, 401...",1.518946,401,-0.650991,422
11,11,"tensor(1753, device='cuda:7')","tensor(indices=tensor([[ 21, 227, 271, 401...",1.425021,401,-0.544832,422
33,33,"tensor(1819, device='cuda:7')","tensor(indices=tensor([[ 121, 322, 394, 401...",1.424217,401,-0.846882,422
26,26,"tensor(745, device='cuda:7')","tensor(indices=tensor([[ 329, 467, 745, 1595...",1.318588,1595,-1.127702,329
22,22,"tensor(1214, device='cuda:7')","tensor(indices=tensor([[ 62, 172, 281, 286...",0.983211,2921,-0.013787,625
10,10,"tensor(1157, device='cuda:7')","tensor(indices=tensor([[ 74, 104, 124, 270...",0.981524,2921,-0.112680,74
27,27,"tensor(2225, device='cuda:7')","tensor(indices=tensor([[ 20, 88, 196, 234...",0.980722,2921,-0.012755,2225


In [27]:
genres_dict = {'0': 'Comedy', '1': 'Drama', '2': 'Romance', '3': 'Thriller'}
for i in topmax[:10].index:
    node_id = topmax['Unnamed: 1'][i].split(',')[0].split('(')[1]
    max_node = topmax.max_nodes[i]
    pos_max_node = topmax.pos_max_nodes[i]
    pred_node_info = overview[overview['id']== int(node_id)]
    uri_pred = nodes[nodes.index == int(node_id)][0].values[0]
    uri_best = nodes[nodes.index == int(pos_max_node)][0].values[0]
    print(uri_pred)
    try:
        job = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['job'].values[0]
        name_best = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['primaryName'].values[0]
    except:
        print('uri_best')
        name_best = movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'].values[0]
        job = 'Year'
        #print('nam:',name_best, movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'] )
    
    mov_title = movies[movies.knownForTitles == uri_pred.split('mov')[1]]['originalTitle'].values[0]
    print('+++++++ STARTING WITH NODE ', mov_title, '+++++++++++++')
    pred_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['pred_label'].values[0])]
    real_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['label'].values[0])]
    print('The movie ', mov_title, 'has the predicted genre ', pred_genre, 'and the real genre ', real_genre, 'because of node ',job, name_best,'.')

    for s, p, o in graph:
        if s == rdf.term.URIRef(uri_pred):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o ==  rdf.term.URIRef(uri_pred):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
        if s ==  rdf.term.URIRef(uri_best):
            if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
                o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
            else:
                print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
        if o == rdf.term.URIRef(uri_best):
            if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
                s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            elif job == 'Year':
                s = 'movie ' + movies[movies.knownForTitles == ('tt' + s.split('/')[-1].split('tt')[1])]['originalTitle'].values[0]
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
            else:
                print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    print('\n')

http://imdb.org/movtt0004902
uri_best
+++++++ STARTING WITH NODE  The Apaches of Paris +++++++++++++
The movie  The Apaches of Paris has the predicted genre  Thriller and the real genre  Drama because of node  Year 1915 .
The  Year 1915 is the following property of: published_in movie What Happened to Jones
The  Year 1915 is the following property of: published_in movie The Bulldogs of the Trail
The  Year 1915 is the following property of: published_in movie Blue Grass
The  Year 1915 is the following property of: published_in movie The Cub
The  Year 1915 is the following property of: published_in movie The Pretenders
The  Year 1915 is the following property of: published_in movie What Happened to Father
The  Year 1915 is the following property of: published_in movie Black Fear
The  Year 1915 is the following property of: published_in movie The Morals of Marcus
The  Year 1915 is the following property of: published_in movie The Little Miss Brown
The  Year 1915 is the following property 

In [509]:
movies[movies.knownForTitles == 'tt0062161']

,nconst,actor,primaryName,actress,knownForTitles,director,tconst,titleType,originalTitle,startYear,Drama,Comedy,Romance,Thriller,genres,job
2810,nm2641247,0,Lei Wan,1,tt0062161,0,tt0062161,movie,Zi bei ke,1967,0,0,1,0,Romance,actress


In [551]:
top_edges = csv[0].sort_values(by=['max_edges'], ascending=False)
top_edges

,Unnamed: 0,Unnamed: 1,tensor_edges,max_edges,pos_max_edges,min_edges,pos_min_edges
16,16,"tensor(2166, device='cuda:7')","tensor(indices=tensor([[ 3, 31, 31, 31...",4.352365,"tensor([[ 56, 2922]])",-3.513505,"tensor([[ 31, 2166]])"
7,7,"tensor(1459, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 3...",3.490031,"tensor([[ 32, 1084]])",-1.551373,"tensor([[ 0, 1938]])"
2,2,"tensor(164, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 2...",2.834977,"tensor([[ 0, 2412]])",-4.812445,"tensor([[ 32, 1084]])"
13,13,"tensor(2867, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 2...",1.831774,"tensor([[ 32, 1084]])",-0.501998,"tensor([[ 0, 2562]])"
11,11,"tensor(2317, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 3...",1.744752,"tensor([[ 32, 1084]])",-0.479311,"tensor([[ 0, 2562]])"
33,33,"tensor(1098, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 3...",1.590636,"tensor([[ 32, 1084]])",-0.433465,"tensor([[ 0, 2562]])"
14,14,"tensor(2814, device='cuda:7')","tensor(indices=tensor([[ 3, 31, 31, 31...",1.577497,"tensor([[ 3, 2861]])",-1.077707,"tensor([[ 56, 2814]])"
0,0,"tensor(1664, device='cuda:7')","tensor(indices=tensor([[ 3, 31, 31, 31...",1.537588,"tensor([[ 3, 2657]])",-1.037203,"tensor([[ 56, 1664]])"
35,35,"tensor(2644, device='cuda:7')","tensor(indices=tensor([[ 3, 31, 31, 31...",1.105121,"tensor([[ 3, 1037]])",-0.605062,"tensor([[ 56, 2644]])"
22,22,"tensor(974, device='cuda:7')","tensor(indices=tensor([[ 0, 1, 2, 2...",0.983061,"tensor([[ 32, 2713]])",-0.004265,"tensor([[ 56, 2576]])"


In [583]:
rel_tab = pd.DataFrame(files[3])
rel_tab

,0,1,2
0,2562.0,0.0,2722.0
1,534.0,0.0,1813.0
2,373.0,1.0,923.0
3,2796.0,2.0,2459.0
4,393.0,3.0,2657.0
...,...,...,...
23784,2924.0,56.0,2924.0
23785,2925.0,56.0,2925.0
23786,2926.0,56.0,2926.0
23787,2927.0,56.0,2927.0


In [585]:
files[2]

['http://imdb.org/ontology/collabs_with',
 'http://imdb.org/ontology/acted_in',
 'http://imdb.org/ontology/has_actor',
 'http://imdb.org/ontology/published_in',
 'http://imdb.org/ontology/preferred_by',
 'http://imdb.org/ontology/directed_by',
 'http://imdb.org/dirnm1024681',
 'http://imdb.org/dirnm0320498',
 'http://imdb.org/dirnm0379602',
 'http://imdb.org/dirnm0529152',
 'http://imdb.org/dirnm5177136',
 'http://imdb.org/dirnm0115450',
 'http://imdb.org/dirnm0603050',
 'http://imdb.org/dirnm0259235',
 'http://imdb.org/dirnm6410027',
 'http://imdb.org/dirnm0028345',
 'http://imdb.org/dirnm0938041',
 'http://imdb.org/dirnm0431344',
 'http://imdb.org/dirnm0790348',
 'http://imdb.org/dirnm0807361',
 'http://imdb.org/dirnm0360114',
 'http://imdb.org/dirnm0201815',
 'http://imdb.org/dirnm0296249',
 'http://imdb.org/dirnm0235121',
 'http://imdb.org/dirnm0085156',
 'http://imdb.org/dirnm10753623',
 'http://imdb.org/dirnm0377053',
 'http://imdb.org/dirnm0233159']

In [577]:
genres_dict = {'0': 'Comedy', '1': 'Drama', '2': 'Romance', '3': 'Thriller'}
for i in top_edges[:10].index:
    node_id = top_edges['Unnamed: 1'][i].split(',')[0].split('(')[1]
    max_edge = top_edges.max_edges[i]
    pos_max_edge = top_edges.pos_max_edges[i].split(',')[0].split(' ')[-1]
    pos_max_node = top_edges.pos_max_edges[i].split(',')[1].split(']')[0]
    print(pos_max_edge)
    pred_node_info = overview[overview['id']== int(node_id)]
    uri_pred = nodes[nodes.index == int(node_id)][0].values[0]
    uri_best = nodes[nodes.index == int(pos_max_node)][0].values[0]
    # print(uri_pred)
    # try:
    #     job = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['job'].values[0]
    #     relation = 
    #     name_best = movies[movies.nconst == ('nm' + uri_best.split('nm')[1])]['primaryName'].values[0]
    # except:
    #     print('uri_best')
    #     name_best = movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'].values[0]
    #     job = 'Year'
    #     #print('nam:',name_best, movies[movies.knownForTitles == ('tt' + uri_pred.split('tt')[-1])]['startYear'] )
    
    # mov_title = movies[movies.knownForTitles == uri_pred.split('mov')[1]]['originalTitle'].values[0]
    # print('+++++++ STARTING WITH NODE ', mov_title, '+++++++++++++')
    # pred_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['pred_label'].values[0])]
    # real_genre = genres_dict[str(test_file[test_file.movie == uri_pred]['label'].values[0])]
    # print('The movie ', mov_title, 'has the predicted genre ', pred_genre, 'and the real genre ', real_genre, 'because of edge ',job, name_best,'.')

    # for s, p, o in graph:
    #     if s == rdf.term.URIRef(uri_pred):
    #         if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
    #             o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
    #             print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
    #         else:
    #             print('The movie ', mov_title, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
    #     if o ==  rdf.term.URIRef(uri_pred):
    #         if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
    #             s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
    #             print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    #         else:
    #             print('The movie ', mov_title, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    #     if s ==  rdf.term.URIRef(uri_best):
    #         if o.split('/')[-1].startswith('act') or o.split('/')[-1].startswith('dir'):
    #             o = movies[movies.nconst == ('nm' + o.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
    #             print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
    #         else:
    #             print('The ', job, name_best, 'has the following properties:', p.split('/')[-1], o.split('/')[-1])
    #     if o == rdf.term.URIRef(uri_best):
    #         if s.split('/')[-1].startswith('act') or s.split('/')[-1].startswith('dir'):
    #             s = movies[movies.nconst == ('nm' + s.split('/')[-1].split('nm')[1])]['primaryName'].values[0]
    #             print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    #         elif job == 'Year':
    #             s = 'movie ' + movies[movies.knownForTitles == ('tt' + s.split('/')[-1].split('tt')[1])]['originalTitle'].values[0]
    #             print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    #         else:
    #             print('The ', job, name_best, 'is the following property of:', p.split('/')[-1], s.split('/')[-1])
    # print('\n')

56
32
0
32
32
32
3
3
3
32


In [ ]:
movies[movies.startYear == 1995].groupby('tconst').first()['genres'].value_counts()

: 

: 